In [55]:
import sys
sys.path.insert(0, '/Users/matthewashman/github/MasterProject2018')

# Import necessary modules. Set settings. Import data.
import math
import numpy as np
import pandas as pd
import random
import pywt
import fastdtw
import matplotlib.pyplot as plt
from statsmodels.robust import mad
from tsfresh.feature_extraction import feature_calculators
from FeatureExtraction.feature_tools import detect_peaks
from IPython.display import display, clear_output, HTML
import re
import seaborn as sns
sns.set(style="whitegrid")

import pdb

plt.style.use('default')

X = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/extracted_segments_with_labels_updated.pkl')

In [56]:
X_S1 = X[X['S1/S2']=='S1']
X_S2 = X[X['S1/S2']=='S2']
X_S2 = X_S2[(X_S2['Label']=='0') |(X_S2['Label']=='1') | (X_S2['Label']=='2')]

## Get Features to be Used In Classification

In [59]:
feature_list = []

for i,row in X_S2.iterrows():
    clear_output(wait=True)
    display('Extracting Features: ' + str(round((i/X_S2.index[-1])*100, 3)) + '%')
    
    # Get the patients response to the first S1 stimuli as the reference response
    # Get typical response for this patient and channel
    # Bad apples
    if (((row['Type'] + row['Patient']) == 'af8') & (row['Channel'] == 'CS5-6')):
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[2]
    elif (((row['Type'] + row['Patient']) == 'at1') & (row['Channel'] == 'CS1-2')):
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[4]
    elif (((row['Type'] + row['Patient']) == 'avnrt10') & (row['Channel'] == 'CS1-2')):
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                           (X_S1['Patient']==row['Patient']) &
                           (X_S1['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[1]
    elif (((row['Type'] + row['Patient']) == 'avrt13') & (row['Channel'] == 'CS1-2')):
        typical_response = X_S2[(X_S2['Type']==row['Type']) & 
                           (X_S2['Patient']==row['Patient']) &
                           (X_S2['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    elif (((row['Type'] + row['Patient']) == 'af14') & (row['Channel'] == 'CS1-2')):
        typical_response = X_S2[(X_S2['Type']==row['Type']) & 
                           (X_S2['Patient']==row['Patient']) &
                           (X_S2['Channel']==row['Channel'])
                           ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
    else:
        typical_response = X_S1[(X_S1['Type']==row['Type']) & 
                               (X_S1['Patient']==row['Patient']) &
                               (X_S1['Channel']==row['Channel'])
                               ].sort_values(by=['Coupling Interval'], ascending=False).iloc[0]
        
    # Normalise amplitudes with respect to the typical response amplitude.
    s1_response = typical_response['Data']/max(abs(typical_response['Data']))
    s2_response = row['Data']/max(abs(typical_response['Data']))
    
    ref_feature_dict = get_feature_dict(s1_response, col_prefix = '')
    feature_dict = get_feature_dict(s2_response, col_prefix = '')
    
    for k, v in feature_dict.items():
        feature_dict[k] = v - ref_feature_dict[k]
        
    
    fdtw = fastdtw.dtw(s2_response, s1_response)
    feature_dict['DTW Distance'] = fdtw[0]
    
    # Fill in the other column values
    for col, value in row.iteritems():
        feature_dict[col] = value
        
    feature_list.append(feature_dict)
    

'Extracting Features: 100.0%'

In [60]:
features = pd.DataFrame(feature_list)

feature_values = features.drop(['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
info = features[['Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

## PCA Projection

In [46]:
%matplotlib qt
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
feature_pc = pca.fit_transform(feature_values.values)
pca_df = pd.DataFrame(data = feature_pc,
                                 columns = ['principal component 1', 'principal component 2'])

final_df = pd.concat([pca_df, info.reset_index()], axis = 1)

labels = ['2', '1','0']
colors = ['r', 'm', 'g']

fig, ax = plt.subplots(figsize=(16,9))

for label, color in zip(labels[::-1], colors[::-1]):
    idx_to_keep = (final_df['Label'].values == label)
    plt.scatter(final_df.loc[idx_to_keep, 'principal component 1']
                   ,final_df.loc[idx_to_keep, 'principal component 2']
                   ,c = color
                   ,edgecolor='k'
                   ,s = 50)
   
names = []
for i, row in final_df.iterrows():
    row_name = row['Type'] + row['Patient'] + ' ' + row['Channel'] + ' ' + row['Coupling Interval'] + ' ' + str(row['Label'])
    names.append(row_name)
    
sc = plt.scatter(final_df['principal component 1'], final_df['principal component 2'],
                alpha=0,
                s=50)
        
annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))

annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}".format(" ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
                
fig.canvas.mpl_connect("motion_notify_event", hover)
plt.legend(['Red', 'Amber', 'Green'])
plt.grid(True)
plt.title('PCA Project of Features')
plt.show()

## LDA Projection Including Augmented Features

In [76]:
X_aug = pd.read_pickle('/Users/matthewashman/github/MasterProject2018/EPDataAnalysis/Final Report/augmented_features_for_lda.pkl')
augmented_features = X_aug[X_aug['Augmented']==1]

augmented_feature_values = augmented_features.drop(['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2'], axis=1)
augmented_info = augmented_features[['Augmented', 'Channel', 'Coupling Interval', 'Data', 'Label', 'Patient', 'Type', 'S1/S2']]

In [92]:
%matplotlib qt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
lda = LDA(n_components=2)
feature_values_lda = lda.fit_transform(feature_values.values, info['Label'])
augmented_feature_values_lda = lda.transform(augmented_feature_values.values)
lda_df = pd.DataFrame(data = feature_values_lda,
                      columns = ['principal component 1', 'principal component 2'])
augmented_lda_df = pd.DataFrame(data = augmented_feature_values_lda,
                                columns = ['principal component 1', 'principal component 2'])

final_df = pd.concat([lda_df, info.reset_index()], axis = 1)
augmented_final_df = pd.concat([augmented_lda_df, augmented_info.reset_index()], axis=1)

labels = ['2', '1','0']
colors = ['r', 'orange', 'g']

fig, ax = plt.subplots(figsize=(12,6))

for label, color in zip(labels[::-1], colors[::-1]):
    idx_to_keep = (final_df['Label'].values == label)
    plt.scatter(final_df.loc[idx_to_keep, 'principal component 1']
                   ,final_df.loc[idx_to_keep, 'principal component 2']
                   ,c = color
                   ,edgecolor='k'
                   ,s = 50)
   
plt.scatter(augmented_final_df['principal component 1'].values, augmented_final_df['principal component 2'].values,
           c='r', 
           edgecolor='k',
           s=50,
           marker='v')
names = []
for i, row in final_df.iterrows():
    row_name = row['Type'] + row['Patient'] + ' ' + row['Channel'] + ' ' + row['Coupling Interval'] + ' ' + str(row['Label'])
    names.append(row_name)
    
sc = plt.scatter(final_df['principal component 1'], final_df['principal component 2'],
                alpha=0,
                s=50)
        
annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))

annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}".format(" ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
                
fig.canvas.mpl_connect("motion_notify_event", hover)
plt.legend(['Green', 'Amber', 'Red', 'Augmented'], fontsize=16)
plt.grid(True)
plt.title('LDA Projection of Features w/ Augmentation', fontsize=16)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.show()

## LDA Projection

In [71]:
%matplotlib qt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn import preprocessing
lda = LDA(n_components=2)
normalised_feature_values = preprocessing.scale(feature_values.values)
feature_values_lda = lda.fit_transform(normalised_feature_values, info['Label'])
lda_df = pd.DataFrame(data = feature_values_lda,
                      columns = ['principal component 1', 'principal component 2'])

final_df = pd.concat([lda_df, info.reset_index()], axis = 1)

labels = ['2', '1','0']
colors = ['r', 'orange', 'g']

fig, ax = plt.subplots(figsize=(12,6))

for label, color in zip(labels[::-1], colors[::-1]):
    idx_to_keep = (final_df['Label'].values == label)
    plt.scatter(final_df.loc[idx_to_keep, 'principal component 1']
                   ,final_df.loc[idx_to_keep, 'principal component 2']
                   ,c = color
                   ,edgecolor='k'
                   ,s = 50)
   
names = []
for i, row in final_df.iterrows():
    row_name = row['Type'] + row['Patient'] + ' ' + row['Channel'] + ' ' + row['Coupling Interval'] + ' ' + str(row['Label'])
    names.append(row_name)
    
sc = plt.scatter(final_df['principal component 1'], final_df['principal component 2'],
                alpha=0,
                s=50)
        
annot = ax.annotate("", xy=(0,0), xytext=(20,20),textcoords="offset points",
                    bbox=dict(boxstyle="round", fc="w"),
                    arrowprops=dict(arrowstyle="->"))

annot.set_visible(False)

def update_annot(ind):

    pos = sc.get_offsets()[ind["ind"][0]]
    annot.xy = pos
    text = "{}".format(" ".join([names[n] for n in ind["ind"]]))
    annot.set_text(text)


def hover(event):
    vis = annot.get_visible()
    if event.inaxes == ax:
        cont, ind = sc.contains(event)
        if cont:
            update_annot(ind)
            annot.set_visible(True)
            fig.canvas.draw_idle()
        else:
            if vis:
                annot.set_visible(False)
                fig.canvas.draw_idle()
                
fig.canvas.mpl_connect("motion_notify_event", hover)
plt.legend(['Green', 'Amber', 'Red'], fontsize=16)
plt.grid(True)
plt.title('LDA Projection of Features', fontsize=16)
plt.yticks(fontsize=14)
plt.xticks(fontsize=14)
plt.show()

In [4]:
# A shitty conduction delay detector
def get_delay(x, amp_thresh=None, set_thresh=False):
    if (set_thresh==True):
        if any(abs(x)>amp_thresh):
            return np.argmax(abs(x)>amp_thresh)
        else:
            return len(x)
    else:    
        return np.argmax(abs(x)>(max(abs(x))/2))
    
def denoise(x):
    # Obtain Daubechies N=6 wavelet coefficients
    waveletCoefs = pywt.wavedec(x, 'db7', mode='per')

    # Throw away coefficients corresponding to noise
    sigma = mad(waveletCoefs[-1])
    uThresh = 1*sigma*np.sqrt(2*np.log(len(x)))
    denoised = waveletCoefs[:]
    denoised[1:] = (pywt._thresholding.hard(i, value=uThresh) for i in denoised[1:])

    # Reconstruct the original signal
    xDenoised = pywt.waverec(denoised, 'db7', mode='per')

    return xDenoised

def get_peaks(x, height_thresh, scale_amp=None, set_scale=False, plot = False):
    x = np.array(x)
    
    # Get height_thresh
    if set_scale:
        height_thresh = height_thresh*scale_amp
    else:
        height_thresh = height_thresh*max(abs(x))
    
    # Denoise x
    xdn = denoise(x)

    # Detect peaks using detect_peaks
    pos_peak_idx = detect_peaks(xdn, mph=height_thresh, threshold = 0)
    neg_peak_idx = detect_peaks((-xdn), mph=height_thresh, threshold = 0)
    peak_idx = np.concatenate([pos_peak_idx, neg_peak_idx])
    peak_idx = np.sort(peak_idx)
    # Edge indeces aren't detected
    peak_idx = peak_idx[(peak_idx != 0) & (peak_idx != (len(xdn)-1))]

    new_peak_idx = []
    peak_amp = []
    if (len(peak_idx) > 0):
        new_peak_idx.append(peak_idx[0])
        mp_thresh = 0.2*max(abs(x))
        for i in range(len(peak_idx)-1):
            idx = peak_idx[i]
            idx_next = peak_idx[i+1]
            mid_point = int((idx_next+idx)/2)
            if (max([abs(x[idx_next]-x[mid_point]), abs(x[idx]-x[mid_point])]) > mp_thresh):
                new_peak_idx.append(idx_next)

        peak_idx = np.array(new_peak_idx)
        peak_amp = x[peak_idx]

    if plot == True:
        fig, [ax1] = plt.subplots(nrows=1, ncols=1, sharex=True, figsize=(8,8))
        ax1.plot(x, 'b' , xdn, 'r--', peak_idx, peak_amp, 'kx')
        #plt.title(fileName)
        ax1.set_xlabel('Sample')
        ax1.set_ylabel('Normalised amplitude')
        ax1.legend(['Original segment', 'Denoised segment', 'Detected peaks'])

        plt.draw()
        plt.waitforbuttonpress(0) # this will wait for indefinite time
        plt.close(fig)


    return peak_idx, peak_amp

def sample_entropy(U, m, r):

    def _maxdist(x_i, x_j):
        result = max([abs(ua-va) for ua, va in zip(x_i, x_j)])
        return result

    def _phi(m):
        x = np.zeros([N,m-1])
        for i in range(N-m+1):
            x[i,:] = U[i:i+m-1]

        C = 0
        for i in range(len(x)):
            for j in range(len(x)):
                if i != j:
                    if _maxdist(x[i,:], x[j,:]) <= r:
                        C = C + 1

        return C

    U = U/max(abs(U))
    N = len(U)

    return -np.log(_phi(m+1)/_phi(m))

def percentage_fractionation(x, peak_idxs, thresh=0.01, sr=1000):
    # Get peak indexes and amplitude
    peak_idx_diffs = np.diff(peak_idxs)
    frac_time = 0
    frac_time = np.sum(peak_idx_diffs[peak_idx_diffs < thresh*sr])
    prcnt_frac = (frac_time/len(x))*100
    return prcnt_frac

def get_local_sample_entropy(x, centre_idx, width, m=2, r=0.05):
    # Ensure width is odd
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return sample_entropy(x[:width+1], m, r)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return sample_entropy(x[len(x)-1-width:], m, r)
    else:
        return sample_entropy(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], m, r)
    
def get_location_of_max_energy(x, M=14):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    return (np.argmax(x_) + math.floor(M/2))
        
def get_local_peaks(x, centre_idx, width=25, height_thresh=0.1):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_peaks(x[:width+1], height_thresh)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_peaks(x[len(x)-1-width:], height_thresh)
    else:
        return get_peaks(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)], height_thresh)
    
def get_pse(x):
    x_fft = np.fft.rfft(x)
    x_P = (1/len(x_fft))*np.absolute(x_fft)**2
    x_p = x_P/sum(x_P)
    pse = np.sum([(-p*np.log2(p)) for p in x_p])
    return pse

def get_local_pse(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_pse(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_pse(x[len(x)-1-width:])
    else:
        return get_pse(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_spectral_centroid(x):
    x_fft = np.fft.rfft(x)
    x_spectrum = np.absolute(x_fft)
    normalized_spectrum = x_spectrum/sum(x_spectrum)
    normalized_frequencies = np.arange(0, len(x_spectrum), 1)
    return sum(normalized_frequencies * normalized_spectrum)

def get_local_spectral_centroid(x, centre_idx, width=50):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return get_spectral_centroid(x[:width+1])
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return get_spectral_centroid(x[len(x)-1-width:])
    else:
        return get_spectral_centroid(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)])
    
def get_local_energy(x, centre_idx, width=60):
    if ((width%2) == 0):
        width += 1
        
    if (centre_idx < (width-1)/2):
        return np.sum(x[:width+1]**2)
    elif (centre_idx > (len(x)-1-(width-1)/2)):
        return np.sum(x[len(x)-1-width:]**2)
    else:
        return np.sum(x[int(centre_idx-(width-1)/2):int(centre_idx+(width+1)/2)]**2)
    
def get_width_max_energy(x, M=14, width_thresh=0.2):
    v = np.ones(M)
    x_ = np.convolve(abs(x), v)
    if any(x_[np.argmax(x_):] < width_thresh*np.max(x_)):
        end_idx = np.argmax(x_) + np.argmax(x_[np.argmax(x_):] < width_thresh*np.max(x_))
    else:
        end_idx = len(x_)-1
    if any(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_)):  
        start_idx = np.argmax(x_) - np.argmax(x_[np.argmax(x_)::-1] < width_thresh*np.max(x_))
    else:
        start_idx = 0

    return (end_idx - start_idx)

In [51]:
def get_feature_dict(x, col_prefix=''):
    feature_dict = {}
    height_thresh=0.1
        
    # Hand engineered features
    peaks = get_peaks(x, height_thresh)
    feature_dict[col_prefix + 'Number of Peaks'] = len(peaks[0])
    feature_dict[col_prefix + 'Percentage Fractionation'] = percentage_fractionation(x, peaks[0], thresh=0.01)
    
    
    max_energy_idx = get_location_of_max_energy(x)
    feature_dict[col_prefix + 'Location of Maximum Energy'] = max_energy_idx
    feature_dict[col_prefix + 'Sample Entropy Around Max Energy'] = get_local_sample_entropy(x, max_energy_idx, 30, m=3, r=0.15)
    feature_dict[col_prefix + 'Width of Maximum Energy'] = get_width_max_energy(x, M=14, width_thresh=0.2)
    
    # Temporal features
    feature_dict[col_prefix + 'Ratio Above 1xSTD'] = feature_calculators.ratio_beyond_r_sigma(x, 1)
    feature_dict[col_prefix + 'Mean Absolute Value'] = np.mean(abs(x)/max(abs(x)))
    
    return feature_dict